In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pip install tensorflow opencv-python moviepy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import os

In [4]:

def extract_frames(video_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count:04d}.jpg"), frame)
        frame_count += 1
    cap.release()

# Example usage
extract_frames(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-real\id0_0000.mp4", "Celeb-real/frames")
extract_frames(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-synthesis\id0_id1_0000.mp4", "Celeb-synthesis/frames")


In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [6]:

def load_images_from_folder(folder, label, image_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return images, labels

real_images, real_labels = load_images_from_folder("Celeb-real/frames", label=0)  # 0 for real
fake_images, fake_labels = load_images_from_folder("Celeb-synthesis/frames", label=1)  # 1 for fake

X = np.array(real_images + fake_images)
y = np.array(real_labels + fake_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
### frames extraction

In [8]:
from moviepy.editor import VideoFileClip

In [9]:
def extract_frames_from_video(video_path, frame_count=30):
    clip = VideoFileClip(video_path)
    frames = []
    duration = clip.duration
    for i in range(frame_count):
        frame = clip.get_frame(i * duration / frame_count)
        frame = cv2.resize(frame, (128, 128))
        frames.append(frame)
    return np.array(frames)

In [10]:
print(extract_frames_from_video(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-real\id0_0000.mp4", 30))

[[[[ 97   0   0]
   [ 95   0   0]
   [ 90   1   0]
   ...
   [ 29  13   7]
   [ 29  16   9]
   [ 33  20  13]]

  [[100   2   1]
   [101   3   2]
   [ 99   1   2]
   ...
   [ 29  13   7]
   [ 29  16   9]
   [ 33  20  13]]

  [[ 86   3   0]
   [ 87   4   0]
   [ 88   2   0]
   ...
   [ 29  13   7]
   [ 29  16   9]
   [ 33  20  13]]

  ...

  [[ 65   0   0]
   [ 65   0   0]
   [ 65   0   0]
   ...
   [ 29  11   6]
   [ 29  11   6]
   [ 29  11   6]]

  [[ 65   0   0]
   [ 65   0   0]
   [ 65   0   0]
   ...
   [ 29  11   6]
   [ 29  11   6]
   [ 29  11   6]]

  [[ 62   0   0]
   [ 62   0   0]
   [ 62   0   0]
   ...
   [ 29  11   6]
   [ 31  10   6]
   [ 31  10   6]]]


 [[[ 96   0   0]
   [ 97   1   1]
   [ 91   0   1]
   ...
   [ 29  13   7]
   [ 31  15   9]
   [ 30  19  12]]

  [[103   1   3]
   [104   2   4]
   [ 99   0   1]
   ...
   [ 29  13   7]
   [ 31  15   9]
   [ 30  19  12]]

  [[ 91   4   1]
   [ 90   3   1]
   [ 87   3   0]
   ...
   [ 29  13   7]
   [ 31  15   9]
   [ 31  20

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

### CNN model

In [12]:
def build_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [13]:
input_shape = (128, 128, 3)  # Adjust based on your frame size
cnn_model = build_cnn_model(input_shape)

In [14]:
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
cnn_model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.3)

Epoch 1/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 183ms/step - accuracy: 0.4877 - loss: 51.4869 - val_accuracy: 0.5333 - val_loss: 0.6909
Epoch 2/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 0.5019 - loss: 0.6996 - val_accuracy: 0.4711 - val_loss: 0.6941
Epoch 3/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.5331 - loss: 0.6921 - val_accuracy: 0.4889 - val_loss: 0.6935
Epoch 4/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - accuracy: 0.5869 - loss: 0.6810 - val_accuracy: 0.5511 - val_loss: 0.6739
Epoch 5/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.5222 - loss: 0.7001 - val_accuracy: 0.5956 - val_loss: 0.6840
Epoch 6/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.5319 - loss: 0.6840 - val_accuracy: 0.5778 - val_loss: 0.6747
Epoch 7/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - accuracy: 0.6037 - loss: 0.6806 - val_accuracy: 0.5556 - val_loss: 0.6659
Epoch 8/40
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 170ms/step - accuracy: 0.5159 - loss: 0.6782 - val_accuracy: 0

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
# model evalution
loss, accuracy = cnn_model.evaluate(X_test, y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 0.0019


In [18]:
print(f'Cnn Model Test Accuracy: {accuracy}')

Cnn Model Test Accuracy: 1.0


### LSTM

In [19]:
from tensorflow.keras.layers import LSTM, TimeDistributed, Bidirectional


In [20]:
# Assuming X_train has shape (num_samples, 128, 128, 3)
# and you want to create sequences of 10 frames
num_frames_per_sequence = 10

# Calculate the number of sequences you can create
num_sequences = len(X_train) // num_frames_per_sequence

# Reshape data into sequences
X_train_lstm = X_train[:num_sequences * num_frames_per_sequence]
X_train_lstm = X_train_lstm.reshape((num_sequences, num_frames_per_sequence, 128, 128, 3))

# Repeat similarly for X_test


In [21]:
from tensorflow.keras import layers

def build_lstm_model(input_shape):
    model = Sequential([
        TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape)),
        TimeDistributed(layers.MaxPooling2D((2, 2))),
        TimeDistributed(layers.Flatten()),
        Bidirectional(layers.LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Bidirectional(layers.LSTM(50, dropout=0.2, recurrent_dropout=0.2)),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [22]:
input_shape = (10, 128, 128, 3)  # Example with 10 frames per sequence
lstm_model = build_lstm_model(input_shape)

In [23]:
# Prepare data for LSTM (e.g., sequences of frames)
# X_train_lstm, X_test_lstm need to be prepared with sequences of frames

# Train the model
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=8, validation_split=0.2)


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.6715 - loss: 0.6574 - val_accuracy: 0.5333 - val_loss: 0.6913
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5186 - loss: 0.6931 - val_accuracy: 0.5333 - val_loss: 0.7147
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5312 - loss: 0.7304 - val_accuracy: 0.5333 - val_loss: 0.7095
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5145 - loss: 0.6952 - val_accuracy: 0.5333 - val_loss: 0.6977
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6066 - loss: 0.6757 - val_accuracy: 0.5333 - val_loss: 0.6909
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6416 - loss: 0.6590 - val_accuracy: 0.5333 - val_loss: 0.6925
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.4251 - loss: 0.7511 - val_accuracy: 0.4667 - val_loss: 0.6950
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.4244 - loss: 0.7202 - val_accuracy: 0.5333 - val_loss: 0.6913
Epoch 9/

Almost 55-60

### Capsule Network

In [24]:
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dense, Lambda, TimeDistributed, Flatten, LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [25]:
## forcing tensorflow to use gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [26]:
## clear previous session
K.clear_session()

In [27]:
## custom squash function for capsule layers
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return s_squared_norm / (1. + s_squared_norm) * x / scale

In [28]:
## capsule network model
def build_capsule_network(input_shape):
    inputs = Input(shape=input_shape)
    conv1 = TimeDistributed(Conv2D(128, (9, 9), activation='relu'))(inputs)  # Reduced filters
    primary_caps = TimeDistributed(Conv2D(8 * 16, (9, 9), strides=2, padding='valid'))(conv1)  # Reduced filters
    primary_caps = TimeDistributed(Reshape((-1, 8)))(primary_caps)
    primary_caps = TimeDistributed(Lambda(squash))(primary_caps)
    
    primary_caps_flat = TimeDistributed(Flatten())(primary_caps)
    lstm_output = LSTM(50)(primary_caps_flat)
    
    caps_output = Dense(1, activation='sigmoid')(lstm_output)
    
    model = Model(inputs=inputs, outputs=caps_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [29]:
X_train = np.random.rand(300, 128, 128, 3)  # Reduced number of frames
y_train = np.random.randint(0, 2, size=(30,))  # Corresponding labels

In [30]:
## reshape the data into sequences of frames
num_frames_per_sequence = 10
num_sequences = len(X_train) // num_frames_per_sequence

X_train_lstm = X_train[:num_sequences * num_frames_per_sequence].reshape(
    (num_sequences, num_frames_per_sequence, 128, 128, 3)
)

y_train_lstm = y_train[:num_sequences]

In [31]:
input_shape = (num_frames_per_sequence, 128, 128, 3)
capsule_model = build_capsule_network(input_shape)

In [32]:
capsule_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 10, 128, 128, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 10, 120, 120, 128)   │          31,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 10, 56, 56, 128)     │       1,327,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 10, 50176, 8)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 10, 50176, 8)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 10, 401408)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 50)                  │      80,291,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 81,650,315 (311.47 MB)

 Trainable params: 81,650,315 (311.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
capsule_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 47s 14s/step - accuracy: 0.3854 - loss: 1.1657 - val_accuracy: 0.6667 - val_loss: 0.6366
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.5625 - loss: 0.7134 - val_accuracy: 0.6667 - val_loss: 0.6423
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 13s/step - accuracy: 0.5938 - loss: 0.6969 - val_accuracy: 0.6667 - val_loss: 0.6366
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 13s/step - accuracy: 0.6250 - loss: 0.6664 - val_accuracy: 0.6667 - val_loss: 0.6366
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 13s/step - accuracy: 0.5781 - loss: 0.6997 - val_accuracy: 0.6667 - val_loss: 0.6365
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.5469 - loss: 0.7213 - val_accuracy: 0.6667 - val_loss: 0.6365
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.6406 - loss: 0.6547 - val_accuracy: 0.6667 - val_loss: 0.6365
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 13s/step - accuracy: 0.6406 - loss: 0.6546 - val_accuracy: 0.6667 - val_loss: 0.6365


In [ ]:
# model evalution
loss, accuracy = capsule_model.evaluate(X_train_lstm, y_train_lstm)
print(f'Cnn Model Test Accuracy: {accuracy}')

### Gans

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LeakyReLU, Dropout

### Generator Model

In [ ]:
latent_dim = 50  # Example latent dimension

# Example generator model that outputs (128, 128, 3) images
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(16 * 16 * 256, input_dim=latent_dim))
    model.add(Reshape((16, 16, 256)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh'))  # (128, 128, 3)
    return model

In [ ]:
# Updated discriminator to accept (128, 128, 3) input shape
def build_discriminator(input_shape=(128, 128, 3)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation

In [ ]:
# Build the generator and discriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator(input_shape=(128, 128, 3))

In [ ]:
# Build the GAN
gan = build_gan(generator, discriminator)

In [ ]:
gan.summary()


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    img = generator(gan_input)
    gan_output = discriminator(img)
    gan = Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    return gan

gan = build_gan(generator, discriminator)


In [ ]:
gan.summary()

In [ ]:
## training of gans

In [ ]:
# Compile the discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile the GAN (keep the discriminator non-trainable during GAN training)
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
def train_gan(gan, generator, discriminator, X_train, latent_dim, n_epochs=10000, n_batch=64, eval_interval=200):
    batch_per_epoch = X_train.shape[0] // n_batch
    half_batch


In [ ]:
import matplotlib.pyplot as plt

def generate_real_samples(X_train, n_samples):
    idx = np.random.randint(0, X_train.shape[0], n_samples)
    X = X_train[idx]
    y = np.ones((n_samples, 1))
    return X, y

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

def save_plot(examples, epoch, n=10):
    examples = (examples + 1) / 2.0  # scale from [-1,1] to [0,1]
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i])
    plt.savefig(f"generated_plot_e{epoch+1}.png")
    plt.close()

def train_gan(gan, generator, discriminator, X_train, latent_dim, n_epochs=10000, n_batch=64, eval_interval=200):
    batch_per_epoch = X_train.shape[0] // n_batch
    half_batch = n_batch // 2
    
    for epoch in range(n_epochs):
        for _ in range(batch_per_epoch):
            X_real, y_real = generate_real_samples(X_train, half_batch)
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss_real = discriminator.train_on_batch(X_real, y_real)
            d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)
            
            x_gan = np.random.randn(latent_dim * n_batch)
            x_gan = x_gan.reshape(n_batch, latent_dim)
            y_gan = np.ones((n_batch, 1))
            g_loss = gan.train_on_batch(x_gan, y_gan)
        
        if (epoch + 1) % eval_interval == 0:
            print(f"Epoch {epoch + 1}/{n_epochs} - D loss: {d_loss_real + d_loss_fake}, G loss: {g_loss}")
            X_fake, _ = generate_fake_samples(generator, latent_dim, 25)
            save_plot(X_fake, epoch)

train_gan(gan, generator, discriminator, X_train, latent_dim)


In [ ]:
## evalution 
d_loss_real, d_acc_real = discriminator.evaluate(X_test, np.ones((X_test.shape[0], 1)))
X_fake, y_fake = generate_fake_samples(generator, latent_dim, X_test.shape[0])
d_loss_fake, d_acc_fake = discriminator.evaluate(X_fake, np.zeros((X_test.shape[0], 1)))

print(f"Discriminator on Real Data - Loss: {d_loss_real}, Accuracy: {d_acc_real}")
print(f"Discriminator on Fake Data - Loss: {d_loss_fake}, Accuracy: {d_acc_fake}")


### Final evalution of all models

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

## CNN Evaluation
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"CNN Loss: {cnn_loss}, Accuracy: {cnn_accuracy}")

## LSTM Evaluation
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_lstm, y_test_lstm)
print(f"LSTM Loss: {lstm_loss}, Accuracy: {lstm_accuracy}")

## capsule Network Evaluation
capsule_loss, capsule_accuracy = capsule_model.evaluate(X_test, y_test)
print(f"Capsule Network Loss: {capsule_loss}, Accuracy: {capsule_accuracy}")

## classification Report and Confusion Matrix
y_pred_cnn = (cnn_model.predict(X_test) > 0.5).astype("int32")
y_pred_lstm = (lstm_model.predict(X_test_lstm) > 0.5).astype("int32")
y_pred_capsule = (capsule_model.predict(X_test) > 0.5).astype("int32")

print("CNN Classification Report:\n", classification_report(y_test, y_pred_cnn))
print("CNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred_cnn))

print("LSTM Classification Report:\n", classification_report(y_test_lstm, y_pred_lstm))
print("LSTM Confusion Matrix:\n", confusion_matrix(y_test_lstm, y_pred_lstm))

print("Capsule Network Classification Report:\n", classification_report(y_test, y_pred_capsule))
print("Capsule Network Confusion Matrix:\n", confusion_matrix(y_test, y_pred_capsule))
